# Modelo de OneClassSVM entrenado con 2F de datos Astro, y predicción sobre datos Unid (no identificados)

**Proyecto**: Detección de posibles fuentes de materia oscura usando ML en datos Fermi-LAT  
**Autor**: Marta Canino Romero  
**Fecha**: febrero-mayo 20225

---

## Descripción:

Este notebook aplica un modelo **One-Class SVM** entrenado con datos de fuentes astrofísicas conocidas (ASTRO) usando las siguientes características:

- E_peak
- beta

Este modelo se entrena para identificar anomalías que puedan corresponder a posibles fuentes de materia oscura (UNIDs) en los datos no identificados del catálogo 4FGL.

---

## Objetivos específicos:

- Entrenar modelo OCSVM con 2 features
- Optimizar hiperparámetros (grid search sobre `nu` y `gamma`)
- Evaluar sobre datos de validación y prueba
- Aplicar modelo final sobre datos UNID para predicción
- Comparar los resultados de anomalía/outliers para OCSVM 2F con los resultados de probabilidad de DM de la ANN 2F (sobre UNIDs)

---

## Entrada de datos:

- `../data/astro_df.txt`

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go

from matplotlib.patches import Rectangle

from plotly.subplots import make_subplots

from scipy.stats import spearmanr, pearsonr

from sklearn.svm import OneClassSVM
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import f1_score, confusion_matrix, classification_report, ConfusionMatrixDisplay, roc_curve, auc
from sklearn.utils.multiclass import unique_labels

from mpl_toolkits.mplot3d import Axes3D

from scipy import stats

import warnings
warnings.filterwarnings('ignore')

# Configurar estilo
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (15, 10)

In [ ]:
# Cargar dataset

# data_path = "../data/astro_df.txt"
data_path = "../data/astro_data_with_labels.txt"

df_astro = pd.read_csv(data_path, sep='\s+')
print(f" Nombres de las columnas: {list(df_astro.columns)}")

df_astro = df_astro.rename(columns={"astro_DM": "class"})

print(f" Dataset cargado. Forma: {df_astro.shape}")
print(f" Nombres de las columnas: {list(df_astro.columns)}")

df_astro.head()

In [ ]:

# Selección de características
features = ['Log(E_peak)', 'Log(beta)']

"""Establecemos la columna objetivo aunque al ser un problema de detección de anomalías no la utilizaremos para entrenar el modelo.
Además de que todos los datos están etiquetados como 'astro_DM' = 0.0"""
target = 'class'


print(f"Features seleccionadas: {features}")
print(f"Columna objetivo: {target}")

# --- Comprobamos valores nulos ---
print("\n Valores faltantes por columna:")
print(df_astro[features + [target]].isnull().sum())

print("\n Muestra del dataset:")
display(df_astro[features + [target]].head())

In [ ]:
# Comprobamos la distribución de los datos astro antes de escalar
# plt.figure(figsize=(10, 8))

sns.scatterplot(
    data=df_astro,
    x="Log(E_peak)",
    y="Log(beta)",
    color="cornflowerblue",  # "skyblue", "turquoise", "cornflowerblue"
    edgecolor='k',
    alpha=0.7,
    s=40
)

plt.title("ASTRO Data: Log(E_peak) vs Log(Beta)")
plt.xlabel("Log(E_peak)")
plt.ylabel("Log(beta)")
plt.grid(True)
plt.show()

In [ ]:

X = df_astro[features].values
y = df_astro[target].values

# SPLIT: Train / Val / Test
# 60% train, 20% val, 20% test
# de esta manera, el test set no se ve en el entrenamiento
# y el val set se usa para ajustar los hiperparámetros)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)


# Escalamos los datos
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)


In [ ]:
print("Forma de X_train_scaled:", X_train_scaled.shape)
print("Forma de X_val_scaled:"  , X_val_scaled.shape)
print("Forma de X_test_scaled:", X_test_scaled.shape)

In [ ]:
# Comprobamos que los datos están bien escalados
# plt.figure(figsize=(10, 8))

sns.scatterplot(
    x=X_train_scaled[:, 0],
    y=X_train_scaled[:, 1],
    color="cornflowerblue",  # "skyblue" "cornflowerblue"
    edgecolor='k',
    alpha=0.7,
    s=40
)
plt.title("Scaled Train Data: Log(E_peak) vs Log(Beta)")
plt.xlabel("Log(E_peak)")
plt.ylabel("Log(beta)")
plt.grid(True)
plt.show()


In [ ]:

"""
PARAMETROS DE OneClassSVM:
    - kernel: rbf (solo cambiar si conocemos la fórmula para la forma de la distribución)
    - gamma: 
        - coeficiente del kernel 
        - usado para hiperplanos no lineares
        - influye en la forma de la frontera de decisión + el desempeño predictivo
        - define la influencia de un único ejemplo de entrenamiento
        - cuanto más grande, más cerca tienen que estar los otros ejemplos para 'verse afectados'
        - (por defecto 'scale' (antes 'auto')) 
    - nu: 
        - límite superior de la fracción de errores permitidos en entrenamiento 
        - límite inferior de la fracción de vectores de soporte en relación con nº total de datos de entrenamiento
            -   ejemplo: si se establece en 0,05 se tiene la garantía de encontrar como máximo el 5 % de los datos de entrenamiento mal clasificados 
                y al menos el 5 % de los datos de entrenamiento siendo vectores de soporte
        - ( (0,1] - 0.5 por defecto )
"""

""" 
EJEMPLOS DE UNDER Y OVERFITTING:

    Underfitting severo
        gamma = 0.001  # muy bajo
        nu = 0.7       # muy alto

    Overfitting severo
        gamma = 10.0    # muy alto
        nu = 0.0001     # extremadamente bajo
"""

"""Hiperparámetros a explorar"""
# gamma_values = ['scale', 'auto'] + list(np.logspace(-3, 1, 5))
gamma_values = [0.1] # Justificación de gamma = 0.1 basada en análisis previo: balance óptimo para que "abrace" los datos sin ser demasiado rígido
nu_values = [0.001, 0.002, 0.005, 0.01, 0.02, 0.05, 0.1, 0.2]
             
"""Tracking"""
results = []
best_outliers = np.inf
best_model = None
best_params = {}
best_mean_score = 0.0

"""Grid Search"""
print("Buscando hiperparámetros que minimicen outliers en ASTRO (validación)...")
for nu in nu_values:
    for gamma in gamma_values:

        model = OneClassSVM(kernel='rbf', nu=nu, gamma=gamma)
        model.fit(X_train_scaled)

        # Calcular score de decisión para análisis más detallado
        decision_scores = model.decision_function(X_val_scaled)
        preds_val = model.predict(X_val_scaled)

        # Predicciones
        preds_val = model.predict(X_val_scaled) # 1 = normal, -1 = outlier
        pred_labels = np.where(preds_val == 1, 0, 1) # Mapear a 0 = normal, 1 = anomalía
        true_labels = y_val.astype(int)
        n_outliers = np.sum(preds_val == -1)
        mean_score = np.mean(decision_scores)  # Distancia media al hiperplano

        # f1 = f1_score(true_labels, pred_labels, pos_label=0)

        results.append({
            'nu': nu, 
            'gamma': gamma, 
            'val_outliers': n_outliers,
            'mean_decision_score': mean_score
        })

        """
        if f1 > best_score:
            best_score = f1
            best_model = model
            best_params = {'nu': nu, 'gamma': gamma}
        """

        if n_outliers < best_outliers:
            best_outliers = n_outliers
            best_model = model
            best_params = {'nu': nu, 'gamma': gamma}
            best_mean_score = mean_score

# Mostrar resultado óptimo
print(f"\n Mejor combinación de hiperparámetros:")
print(f"   - ν = {best_params['nu']}")
print(f"   - γ = {best_params['gamma']}")
print(f"   - Outliers (val set): {best_outliers} de {len(X_val_scaled)} muestras")
print(f"   - Mejor distancia media al hiperplano: {best_mean_score} ")

df_results = pd.DataFrame(results)
display(df_results.sort_values(by='val_outliers'))


In [ ]:
"""# Evaluar el modelo en el conjunto de validación
val_preds = best_model.predict(X_val_scaled)
n_val_outliers = np.sum(val_preds == -1)
print(f"Outliers (val set): {n_val_outliers} de {len(X_val_scaled)} muestras")

# Evaluar el modelo en el conjunto de test
# (no se ha visto en el entrenamiento)
test_preds = best_model.predict(X_test_scaled)
n_test_outliers = np.sum(test_preds == -1)
print(f"Outliers (test set): {n_test_outliers} de {len(X_test_scaled)} muestras")"""

In [ ]:

# Reentrenar con X_train + X_val
X_final_train = np.vstack([X_train, X_val])
y_final_train = np.concatenate([y_train, y_val])

scaler_final = StandardScaler()

X_final_train_scaled = scaler_final.fit_transform(X_final_train)

X_test_scaled = scaler_final.transform(X_test)

# Entrenar modelo final
final_model = OneClassSVM(kernel='rbf', gamma=best_params['gamma'], nu=best_params['nu'])
final_model.fit(X_final_train_scaled)

In [ ]:

# Análisis de la distribución de scores
decision_scores_test = final_model.decision_function(X_test_scaled)

print(f"Decision scores estadísticas:")
print(f"  Media: {np.mean(decision_scores_test):.4f}")
print(f"  Std: {np.std(decision_scores_test):.4f}")
print(f"  Min: {np.min(decision_scores_test):.4f}")
print(f"  Max: {np.max(decision_scores_test):.4f}")

In [ ]:

# Evaluar en test
test_preds = final_model.predict(X_test_scaled)
test_labels = np.where(test_preds == 1, 0, 1)

print("Evaluación en el conjunto de test:")
print(f"\nOutliers (test set): {np.sum(test_preds == -1)} de {len(X_test_scaled)} muestras")

In [ ]:
# Metricas
print("Matriz de confusión:")
cm = confusion_matrix(y_test, test_labels)
print(cm)

print(f"\nReporte de clasificación:")
print(classification_report(y_test, test_labels, target_names=unique_labels(y_test, test_labels).astype(str)))

In [ ]:
# Analizar qué características tienen los "outliers" detectados
outlier_indices = np.where(test_preds == -1)[0]
if len(outlier_indices) > 0:
    print(f"\nCaracterísticas de los outliers detectados:")
    outlier_features = X_test[outlier_indices]
    print(f"Log(E_peak) range: [{np.min(outlier_features[:, 0]):.3f}, {np.max(outlier_features[:, 0]):.3f}]")
    print(f"Log(beta) range: [{np.min(outlier_features[:, 1]):.3f}, {np.max(outlier_features[:, 1]):.3f}]")

In [ ]:
# Generar la malla para la frontera de decisión
xx, yy = np.meshgrid(
    np.linspace(X_final_train_scaled[:, 0].min() - 0.5, X_final_train_scaled[:, 0].max() + 0.5, 300),
    np.linspace(X_final_train_scaled[:, 1].min() - 0.5, X_final_train_scaled[:, 1].max() + 0.5, 300)
)

grid = np.c_[xx.ravel(), yy.ravel()]
Z = final_model.decision_function(grid)
Z = Z.reshape(xx.shape)

# Crear la figura
plt.figure(figsize=(12, 8))

# Contornos de la función de decisión - GUARDAR LA REFERENCIA
contour_plot = plt.contourf(xx, yy, Z, levels=np.linspace(Z.min(), 0, 7), cmap=plt.cm.PuBu_r, alpha=0.8)
plt.contour(xx, yy, Z, levels=[0], linewidths=2, colors='crimson', linestyles='--')

# Añadir barra de color usando la referencia
cbar = plt.colorbar(contour_plot)
cbar.set_label('Decision Function Value', rotation=270, labelpad=20)

# Plotear datos de entrenamiento y test
plt.scatter(X_test_scaled[:, 0], X_test_scaled[:, 1], 
           c='darkblue', edgecolors='white', s=50, alpha=0.8,
           label='ASTRO Test Data Sources')
plt.scatter(X_final_train_scaled[:, 0], X_final_train_scaled[:, 1], 
           c='skyblue', edgecolors='white', s=50, alpha=0.8,
           label='ASTRO Train Data Sources')

# Etiquetas y título con parámetros
plt.xlabel("Log(E_peak) (scaled)", fontsize=12)
plt.ylabel("Log(beta) (scaled)", fontsize=12)

# Título dinámico con parámetros del modelo
gamma_val = final_model.gamma if isinstance(final_model.gamma, (int, float)) else final_model.gamma
title = f"One-Class SVM Decision Boundary\nγ = {gamma_val}, ν = {final_model.nu}"
plt.title(title, fontsize=14, fontweight='bold')

# Leyenda y grid
plt.legend(loc='upper right', framealpha=0.9)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Información adicional del modelo
print(f"Modelo One-Class SVM:")
print(f"  - Kernel: {final_model.kernel}")
print(f"  - Gamma: {final_model.gamma}")
print(f"  - Nu: {final_model.nu}")
print(f"  - Número de vectores soporte: {final_model.n_support_[0]}")
print(f"  - Proporción de outliers en entrenamiento: {(final_model.predict(X_final_train_scaled) == -1).mean():.3f}")

# Importar UNIDs

In [ ]:

# Cargar datos UnIDs

# unids_path = "../data/unids_log.txt"
unids_path = "../data/unids_transformed_complete.txt"

# df_unids = pd.read_csv(unids_path, sep='\s+')
df_unids = pd.read_csv(unids_path, sep='\t')

print("Datos UnIDs cargados:")
print(f"Shape: {df_unids.shape}")
print("Primeras filas:")
display(df_unids.head())

# Verificar columnas disponibles
print(f"Columnas disponibles: {list(df_unids.columns)}")

In [ ]:

# Extraer y escalar 
X_unids_log = df_unids[["Log(E_peak)", "Log(beta)"]].values
X_unids_scaled = scaler.transform(X_unids_log)

In [ ]:
# Comparar rangos con datos de entrenamiento para detectar extrapolación
train_range_epeak = [X_train[:, 0].min(), X_train[:, 0].max()]
train_range_beta = [X_train[:, 1].min(), X_train[:, 1].max()]

unids_range_epeak = [X_unids_log[:, 0].min(), X_unids_log[:, 0].max()]
unids_range_beta = [X_unids_log[:, 1].min(), X_unids_log[:, 1].max()]

print(f"Rangos de datos:")
print(f"  Train E_peak: {train_range_epeak}")
print(f"  UNIDs E_peak: {unids_range_epeak}")
print(f"  Train beta: {train_range_beta}")
print(f"  UNIDs beta: {unids_range_beta}")

In [ ]:
# --- Crear subplots 3x2 ---
fig, axes = plt.subplots(3, 2, figsize=(16, 18))

# --- Plot 1: UNIDs sin escalar ---
sns.scatterplot(
    data=df_unids,
    x="Log(E_peak)",
    y="Log(beta)",
    color="gold",
    edgecolor='k',
    alpha=0.7,
    s=40,
    ax=axes[0, 0]
)
axes[0, 0].set_title("UNIDs Data (sin escalar): Log(E_peak) vs Log(β)")
axes[0, 0].set_xlabel("Log(E_peak)")
axes[0, 0].set_ylabel("Log(beta)")
axes[0, 0].grid(True)

# --- Plot 2: UNIDs escalados ---
sns.scatterplot(
    x=X_unids_scaled[:, 0],
    y=X_unids_scaled[:, 1],
    color="gold",
    edgecolor='k',
    alpha=0.7,
    s=40,
    ax=axes[0, 1]
)
axes[0, 1].set_title("UNIDs Escalados: Log(E_peak) vs Log(β)")
axes[0, 1].set_xlabel("Log(E_peak) (escalado)")
axes[0, 1].set_ylabel("Log(beta) (escalado)")
axes[0, 1].grid(True)

# --- Plot 3: Astro sin escalar ---
sns.scatterplot(
    data=df_astro,  # asegúrate de tener este DataFrame cargado
    x="Log(E_peak)",
    y="Log(beta)",
    color="skyblue",
    edgecolor='k',
    alpha=0.7,
    s=40,
    ax=axes[1, 0]
)
axes[1, 0].set_title("Astro Data (sin escalar): Log(E_peak) vs Log(β)")
axes[1, 0].set_xlabel("Log(E_peak)")
axes[1, 0].set_ylabel("Log(beta)")
axes[1, 0].grid(True)

# --- Plot 4: Astro escalado ---
sns.scatterplot(
    x=X_final_train_scaled[:, 0],
    y=X_final_train_scaled[:, 1],
    color="skyblue",
    edgecolor='k',
    alpha=0.7,
    s=40,
    ax=axes[1, 1]
)
axes[1, 1].set_title("Astro Escalado: Log(E_peak) vs Log(β)")
axes[1, 1].set_xlabel("Log(E_peak) (escalado)")
axes[1, 1].set_ylabel("Log(beta) (escalado)")
axes[1, 1].grid(True)

# --- Plot 5: Comparación UNIDs vs Astro escalados ---
sns.scatterplot(
    x=X_final_train_scaled[:, 0],
    y=X_final_train_scaled[:, 1],
    color="skyblue",
    edgecolor='k',
    alpha=0.7,
    s=40,
    label='Astro',
    ax=axes[2, 0]
)
sns.scatterplot(
    x=X_unids_scaled[:, 0],
    y=X_unids_scaled[:, 1],
    color="gold",
    edgecolor='k',
    alpha=0.7,
    s=40,
    label='UNIDs',
    ax=axes[2, 0]
)
axes[2, 0].set_title("UNIDs vs Astro Escalados: Log(E_peak) vs Log(β)")
axes[2, 0].set_xlabel("Log(E_peak) (escalado)")
axes[2, 0].set_ylabel("Log(beta) (escalado)")
axes[2, 0].legend()
axes[2, 0].grid(True)

# --- Eliminar celda vacía (3,2) ---
fig.delaxes(axes[2, 1])

# --- Layout final ---
plt.tight_layout()
plt.show()


# Predecir sobre UNIDs

In [ ]:

""" Predecir sobre UNIDs usando el mejor modelo """
""" IMPORTANTE: ESTOS PARÁMETROS SE USAN EN LOS PLOTS --- NO TOCAR """

decision_scores = final_model.decision_function(X_unids_scaled)

# Top 5 most anomalous
"""top_anomalies = X_unids_scaled[np.argsort(decision_scores)[:5]]"""
preds_unids = final_model.predict(X_unids_scaled)

n_outliers = np.sum(preds_unids == -1)
n_inliers = np.sum(preds_unids == 1)
total_unids = len(preds_unids)

# Para gráficas
outlier_percentage = n_outliers / total_unids * 100
inlier_percentage = n_inliers / total_unids * 100

inliers = X_unids_scaled[preds_unids == 1]    # Para scatter gold
outliers = X_unids_scaled[preds_unids == -1]  # Para scatter red
top_anomalies = X_unids_scaled[np.argsort(decision_scores)[:5]]  # Para stars

In [ ]:

print(f"RESULTADOS DE PREDICCIÓN:")
print(f"  - Total UNIDs: {total_unids}")
print(f"  - Outliers: {n_outliers} ({n_outliers/total_unids*100:.1f}%)")
print(f"  - Inliers: {n_inliers} ({n_inliers/total_unids*100:.1f}%)")

print(f"Outliers detectados con sus IDs:")

outlier_ids = df_unids[preds_unids == -1].index.tolist()
print(outlier_ids)

# Calcular anomaly scores y rankings
# Invertir scores: valores más altos = más anómalos
anom_scores = -decision_scores

# Escalar a percentiles [0, 100]
anom_percent = MinMaxScaler(feature_range=(0, 100)).fit_transform(
    anom_scores.reshape(-1, 1)
).flatten()

# Agregar resultados al DataFrame
df_unids_results = df_unids.copy()
df_unids_results["svm_score"] = decision_scores
df_unids_results["prediction"] = preds_unids
df_unids_results["Anomaly_Score"] = anom_scores
df_unids_results["Anomaly_Rank(%)"] = anom_percent

print("Scores calculados y agregados al DataFrame")

In [ ]:

# Top candidatos
outliers_only = df_unids_results[df_unids_results["prediction"] == -1].copy()
top_10_outliers = outliers_only.sort_values("Anomaly_Rank(%)", ascending=False).head(10)

In [ ]:

outliers_only = outliers_only.sort_values("Anomaly_Rank(%)", ascending=False)
display(outliers_only.head(10))

In [ ]:
# Comprobamos los valores mínimos y máximos de astro_df
"""
print("\nValores mínimos y máximos de las características en astro_df:")
for feature in features:
    min_val = df_astro[feature].min()
    max_val = df_astro[feature].max()
    print(f"{feature}: [{min_val:.4f}, {max_val:.4f}]")
    """
# Comprobamos los valores mínimos y máximos de unids
print("\nValores mínimos y máximos de las características en unids:")
for feature in df_unids.columns:
    min_val = df_unids[feature].min()
    max_val = df_unids[feature].max()
    print(f"{feature}: [{min_val:.4f}, {max_val:.4f}]")

In [ ]:
df_unids_results.head()

In [ ]:
""" Obtener frontera de decisión del entrenamiento del modelo """
xx, yy = np.meshgrid(
    np.linspace(X_train_scaled[:, 0].min() - 0.5, X_train_scaled[:, 0].max() + 0.5, 300),  # E_peak
    np.linspace(X_train_scaled[:, 1].min() - 0.5, X_train_scaled[:, 1].max() + 0.5, 300)   # beta
)
"""  Establecer frontera """
grid = np.c_[xx.ravel(), yy.ravel()]
Z = final_model.decision_function(grid)
Z = Z.reshape(xx.shape)

""" Definir plot """
plt.figure(figsize=(10, 8))
plt.contourf(xx, yy, Z, levels=np.linspace(Z.min(), 0, 7), cmap=plt.cm.PuBu_r, alpha=0.8)
plt.contour(xx, yy, Z, levels=[0], linewidths=2, colors='crimson', linestyles='--')

"""" Plot scatter data """
plt.scatter(inliers[:, 0], inliers[:, 1], 
           c='gold', edgecolors='black', s=50, alpha=0.7,
           label=f'UNID Inliers ({len(inliers)})')
           
plt.scatter(outliers[:, 0], outliers[:, 1], 
           c='red', edgecolors='black', s=60, alpha=0.9,
           label=f'UNID Outliers ({len(outliers)})')
           
plt.scatter(top_anomalies[:, 0], top_anomalies[:, 1], 
           c='darkred', edgecolors='white', s=100, 
           label='Top Anomalies', marker='*', linewidth=1)

plt.xlabel("E_peak (scaled)")
plt.ylabel("beta (scaled)")
plt.title(f"OneClassSVM Predictions on {total_unids} UNIDs\n"
          f"Outliers: {n_outliers} ({outlier_percentage:.1f}%) | "
          f"γ={best_params['gamma']}, ν={best_params['nu']}")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
plt.figure(figsize=(10, 8))
plt.scatter(inliers[:, 0], inliers[:, 1], c='gold', edgecolors='k', label='Inlier (likely astro)', alpha=0.6)
plt.scatter(outliers[:, 0], outliers[:,1], c='red', edgecolors='k', label='Outlier (potentially new)', alpha=0.6)

plt.xlabel("Log(E_peak) (scaled)")
plt.ylabel("Log(beta) (scaled)")
plt.title("UNIDs: Inlier vs Outlier (One-Class SVM)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

## Anomaly Scoring - UNIDs

In [ ]:

""" Predecir sobre UNIDs usando el mejor modelo """
""" IMPORTANTE: ESTOS PARÁMETROS SE USAN EN LOS PLOTS --- NO TOCAR """

decision_scores = final_model.decision_function(X_unids_scaled)
# Top 5 most anomalous
top_anomalies = X_unids_scaled[np.argsort(decision_scores)[:5]]
preds_unids = final_model.predict(X_unids_scaled)
total_unids = len(preds_unids)

# Contar resultados
inliers = X_unids_scaled[preds_unids == 1]
outliers = X_unids_scaled[preds_unids == -1]

n_outliers = np.sum(preds_unids == -1)
n_inliers = np.sum(preds_unids == 1)

outlier_percentage = n_outliers / total_unids * 100
inlier_percentage = n_inliers / total_unids * 100

In [ ]:

# Filtrar solo outliers y ordenar por anomaly rank
outliers_only = df_unids_results[df_unids_results["prediction"] == -1].copy()
outliers_sorted = outliers_only.sort_values(by="Anomaly_Rank(%)", ascending=False)

# Top 10 outliers más anómalos
top_10_outliers = outliers_sorted.head(10)

print(f"\nTOP 10 CANDIDATOS MÁS ANÓMALOS:")

# Mostrar información relevante
display_cols = ['Log(E_peak)', 'Log(beta)', 'Log(sigma)', 'Log(beta_Rel)', 
                'svm_score', 'Anomaly_Score', 'Anomaly_Rank(%)']

if 'number' in df_unids_results.columns:
    display_cols = ['number'] + display_cols

display(top_10_outliers[display_cols].round(4))

In [ ]:
# Configuración de la gráfica
plt.figure(figsize=(12, 8))

# Separar inliers y outliers
inliers_mask = preds_unids == 1
outliers_mask = preds_unids == -1

# Plot inliers (usando las dos primeras features: Log(E_peak) vs Log(beta))
plt.scatter(X_unids_scaled[inliers_mask, 0], X_unids_scaled[inliers_mask, 1], 
           c='lightblue', s=25, alpha=0.6, edgecolors='blue', linewidth=0.3,
           label=f'Inliers (astro-like): {np.sum(inliers_mask)}')

# Plot outliers con gradient de color según anomaly rank
if np.any(outliers_mask):
    scatter = plt.scatter(X_unids_scaled[outliers_mask, 0], X_unids_scaled[outliers_mask, 1], 
                         c=anom_percent[outliers_mask], cmap='Reds', s=80,
                         alpha=0.9, edgecolors='black', linewidth=0.8,
                         label=f'Outliers (anomalous): {np.sum(outliers_mask)}')
    
    # Colorbar
    cbar = plt.colorbar(scatter, label='Anomaly Rank %')
    cbar.ax.tick_params(labelsize=10)

# Etiquetar TODOS los outliers con sus IDs
if np.any(outliers_mask):
    outlier_indices = np.where(outliers_mask)[0]
    
    for idx in outlier_indices:
        x = X_unids_scaled[idx, 0]  # Log(E_peak)
        y = X_unids_scaled[idx, 1]  # Log(beta)
        
        # Obtener el ID del UNID
        if 'number' in df_unids_results.columns:
            unid_id = int(df_unids_results.loc[idx, 'number'])
        else:
            unid_id = idx
        
        # Calcular offset para evitar solapamientos
        offset_x = (X_unids_scaled[:, 0].max() - X_unids_scaled[:, 0].min()) * 0.015
        offset_y = (X_unids_scaled[:, 1].max() - X_unids_scaled[:, 1].min()) * 0.015
        
        # Etiqueta con caja para mejor legibilidad
        plt.text(x + offset_x, y + offset_y, str(unid_id), 
                fontsize=8, color='black', fontweight='bold',
                ha='left', va='bottom',
                bbox=dict(boxstyle='round,pad=0.2', facecolor='white', 
                         alpha=0.8, edgecolor='gray', linewidth=0.5))

# Destacar top 10 anomalías con círculos amarillos
if 'top_10_outliers' in locals() and len(top_10_outliers) > 0:
    top_10_indices = top_10_outliers.index
    plt.scatter(X_unids_scaled[top_10_indices, 0], X_unids_scaled[top_10_indices, 1],
               s=150, facecolors='none', edgecolors='yellow', linewidth=3,
               label=f'Top Anomalies', marker='o')

# Configuración de la gráfica
plt.xlabel('Log(E_peak)', fontsize=12)
plt.ylabel('Log(beta)', fontsize=12)
plt.title(f'UNIDs: Detección de Outliers ({np.sum(outliers_mask)} detectados)\n'
          f'Log(E_peak) vs Log(beta) - Gradient por Anomaly Rank',
          fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)

# Ajustar límites para mejor visualización de etiquetas
x_margin = (X_unids_scaled[:, 0].max() - X_unids_scaled[:, 0].min()) * 0.1
y_margin = (X_unids_scaled[:, 1].max() - X_unids_scaled[:, 1].min()) * 0.1
plt.xlim(X_unids_scaled[:, 0].min() - x_margin, X_unids_scaled[:, 0].max() + x_margin)
plt.ylim(X_unids_scaled[:, 1].min() - y_margin, X_unids_scaled[:, 1].max() + y_margin)

plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Análisis Estadístico del Modelo OCSVM en UNIDs', fontsize=16, fontweight='bold', y=0.98)

# 1. Distribución de Decision Scores CON ESTADÍSTICAS
ax1 = axes[0, 0]
n, bins, patches = ax1.hist(decision_scores, bins=50, alpha=0.7, color='skyblue', 
                           edgecolor='black', density=True)

# Añadir curva de densidad suavizada
x_smooth = np.linspace(decision_scores.min(), decision_scores.max(), 100)
density = stats.gaussian_kde(decision_scores)
ax1.plot(x_smooth, density(x_smooth), 'navy', linewidth=2, label='Densidad estimada')

# Líneas de referencia
ax1.axvline(0, color='red', linestyle='--', linewidth=2, label='Frontera (score=0)')
ax1.axvline(decision_scores.mean(), color='green', linestyle='--', linewidth=2,
           label=f'Media: {decision_scores.mean():.3f}')
ax1.axvline(np.median(decision_scores), color='orange', linestyle='--', linewidth=2,
           label=f'Mediana: {np.median(decision_scores):.3f}')

# Estadísticas adicionales
std_dev = decision_scores.std()
ax1.fill_betweenx([0, ax1.get_ylim()[1]], decision_scores.mean()-std_dev, 
                  decision_scores.mean()+std_dev, alpha=0.2, color='green', 
                  label=f'±1σ: {std_dev:.3f}')

ax1.set_xlabel('SVM Decision Score')
ax1.set_ylabel('Densidad de Probabilidad')
ax1.set_title('Distribución de Decision Scores\n(Análisis de Normalidad)')
ax1.legend(fontsize=9)
ax1.grid(True, alpha=0.3)

# Texto con estadísticas
stats_text = f'n = {len(decision_scores)}\nSkewness: {stats.skew(decision_scores):.3f}\nKurtosis: {stats.kurtosis(decision_scores):.3f}'
ax1.text(0.02, 0.98, stats_text, transform=ax1.transAxes, fontsize=9, 
         verticalalignment='top', bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

# 2. Distribución de Anomaly Ranks CON PERCENTILES
ax2 = axes[0, 1]
ax2.hist(anom_percent, bins=50, alpha=0.7, color='lightcoral', edgecolor='black')

# Percentiles importantes
percentiles = [50, 75, 90, 95, 99]
colors = ['blue', 'orange', 'red', 'darkred', 'purple']
for p, color in zip(percentiles, colors):
    value = np.percentile(anom_percent, p)
    ax2.axvline(value, color=color, linestyle='--', linewidth=2, 
               label=f'P{p}: {value:.1f}%')

ax2.set_xlabel('Anomaly Rank (%)')
ax2.set_ylabel('Frecuencia')
ax2.set_title('Distribución de Anomaly Rankings\n(Análisis de Percentiles)')
ax2.legend(fontsize=9, loc='upper left')
ax2.grid(True, alpha=0.3)

# 3. Comparación Inliers vs Outliers
ax3 = axes[1, 0]
outliers_scores = anom_percent[preds_unids == -1]
inliers_scores = anom_percent[preds_unids == 1]

# Histograma
bins = np.linspace(0, 100, 31)
ax3.hist([inliers_scores, outliers_scores], bins=bins, alpha=0.7,
         color=['lightblue', 'red'], label=[f'Inliers (n={len(inliers_scores)})', 
                                           f'Outliers (n={len(outliers_scores)})'],
         edgecolor='black')

# Añadir líneas de medias
ax3.axvline(inliers_scores.mean(), color='blue', linestyle='-', linewidth=2,
           label=f'Media Inliers: {inliers_scores.mean():.1f}%')
ax3.axvline(outliers_scores.mean(), color='darkred', linestyle='-', linewidth=2,
           label=f'Media Outliers: {outliers_scores.mean():.1f}%')

ax3.set_xlabel('Anomaly Rank (%)')
ax3.set_ylabel('Frecuencia')
ax3.set_title('Separación Inliers vs Outliers\n(Validación del Modelo)')
ax3.legend(fontsize=9)
ax3.grid(True, alpha=0.3)

# Test estadístico
stat, p_value = stats.mannwhitneyu(outliers_scores, inliers_scores, alternative='greater')
test_text = f'Mann-Whitney U test:\np-value: {p_value:.2e}'
ax3.text(0.02, 0.98, test_text, transform=ax3.transAxes, fontsize=9,
         verticalalignment='top', bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

# 4. Scatter con regiones de interés
ax4 = axes[1, 1]
colors = ['red' if pred == -1 else 'blue' for pred in preds_unids]
sizes = [40 if pred == -1 else 20 for pred in preds_unids]

scatter = ax4.scatter(decision_scores, anom_percent, c=colors, alpha=0.6, s=sizes,
                     edgecolors='black', linewidth=0.5)

# Regiones de interés
ax4.axvline(0, color='black', linestyle='--', alpha=0.7, linewidth=2, label='Frontera decisión')
ax4.axhline(90, color='red', linestyle='--', alpha=0.7, label='Top 10% anomalías')
ax4.axhline(95, color='darkred', linestyle='--', alpha=0.7, label='Top 5% anomalías')

# Destacar zona de interés (outliers extremos)
extreme_mask = (preds_unids == -1) & (anom_percent > 95)
if np.any(extreme_mask):
    ax4.scatter(decision_scores[extreme_mask], anom_percent[extreme_mask], 
               s=100, facecolors='none', edgecolors='yellow', linewidth=3,
               label=f'Outliers extremos (n={np.sum(extreme_mask)})')

# Correlación
correlation = np.corrcoef(decision_scores, anom_percent)[0, 1]
ax4.text(0.02, 0.98, f'Correlación: {correlation:.3f}', transform=ax4.transAxes, 
         fontsize=11, verticalalignment='top', fontweight='bold',
         bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.8))

ax4.set_xlabel('SVM Decision Score')
ax4.set_ylabel('Anomaly Rank (%)')
ax4.set_title('Relación Decision Score vs Anomaly Rank\n(Coherencia del Modelo)')
ax4.legend(fontsize=9, loc='lower right')
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Save model for API extra

In [ ]:
# Export trained model to disk
import joblib
model_path = "../models/model_ocsvm_2f.pkl"
joblib.dump(final_model, model_path)
print(f"Modelo One-Class SVM guardado en: {model_path}")

# OCSVM 2F vs ANN 2F - Comparar los resultados (sobre UNIDs) con los de la red neuronal

In [ ]:
# ANÁLISIS COMPARATIVO: ANN 2F vs OCSVM 2F

print("ANÁLISIS COMPARATIVO: ANN 2F vs OCSVM 2F\n")

# Cargar resultados de consenso ANN 2F
ann_consensus_path = "../data/results/ann/2F/consensus_analysis_ann_2f_improved.csv"
df_ann_consensus = pd.read_csv(ann_consensus_path)

print(f"Datos ANN 2F cargados:")
print(f"  - Shape: {df_ann_consensus.shape}")
print(f"  - Columnas principales: Source_ID, Mean_Consensus, Std_Consensus\n")

df_ann_consensus.head()

In [ ]:
# Obtener candidatos OCSVM 2F (outliers detectados)
ocsvm_outliers = set()
if np.any(preds_unids == -1):
    outlier_indices = np.where(preds_unids == -1)[0]
    for idx in outlier_indices:
        if 'number' in df_unids_results.columns:
            unid_id = int(df_unids_results.loc[idx, 'number'])
            ocsvm_outliers.add(unid_id)

# Top 10 OCSVM por anomaly rank
ocsvm_top10 = set()
if len(df_unids_results) > 0:
    top_10_ocsvm = df_unids_results.nlargest(10, 'Anomaly_Rank(%)')
    for idx, row in top_10_ocsvm.iterrows():
        if 'number' in row:
            ocsvm_top10.add(int(row['number']))

print(f"Candidatos OCSVM 2F:")
print(f"  - Outliers detectados: {len(ocsvm_outliers)} → {sorted(ocsvm_outliers)}")
print(f"  - Top 10 anomalías: {len(ocsvm_top10)} → {sorted(ocsvm_top10)}\n")

In [ ]:
# Obtener candidatos ANN 2F
ann_top4 = set(df_ann_consensus.head(4)['Source_ID'].astype(int))
ann_top10 = set(df_ann_consensus.head(10)['Source_ID'].astype(int))

# Candidatos ANN por threshold de consenso
consensus_threshold = 0.5
high_consensus_mask = df_ann_consensus['Mean_Consensus'] >= consensus_threshold
ann_high_consensus = set()
if high_consensus_mask.any():
    ann_high_consensus = set(df_ann_consensus[high_consensus_mask]['Source_ID'].astype(int))

print(f"Candidatos ANN 2F:")
print(f"  - Top 4: {sorted(ann_top4)}")
print(f"  - Top 10: {sorted(ann_top10)}")
print(f"  - Alto consenso (≥{consensus_threshold}): {len(ann_high_consensus)} → {sorted(ann_high_consensus)}\n")


In [ ]:
# VERIFICACIÓN DE CONSISTENCIA DE DATOS
print("VERIFICACIÓN DE CONSISTENCIA DE DATOS\n")

# Obtener algunos candidatos para verificar
verification_candidates = list(ocsvm_outliers.union(ann_top4))[:5]  # Primeros 5 para verificar

print("Verificando que Source_ID (ANN) = number (OCSVM) corresponden a las mismas fuentes:\n")

consistent_data = True
for candidate in verification_candidates:
    # Buscar en datos ANN
    ann_row = df_ann_consensus[df_ann_consensus['Source_ID'] == candidate]
    # Buscar en datos OCSVM
    ocsvm_row = df_unids_results[df_unids_results['number'] == candidate]
    
    if len(ann_row) > 0 and len(ocsvm_row) > 0:
        # Extraer valores de features
        ann_epeak = ann_row['log_E_peak'].iloc[0] if 'log_E_peak' in ann_row.columns else None
        ann_beta = ann_row['log_Beta'].iloc[0] if 'log_Beta' in ann_row.columns else None
        
        ocsvm_epeak = ocsvm_row['Log(E_peak)'].iloc[0] if 'Log(E_peak)' in ocsvm_row.columns else None
        ocsvm_beta = ocsvm_row['Log(beta)'].iloc[0] if 'Log(beta)' in ocsvm_row.columns else None
        
        if ann_epeak is not None and ocsvm_epeak is not None:
            epeak_diff = abs(ann_epeak - ocsvm_epeak)
            beta_diff = abs(ann_beta - ocsvm_beta)
            
            # Tolerancia para diferencias de precisión numérica
            tolerance = 1e-6
            epeak_match = epeak_diff < tolerance
            beta_match = beta_diff < tolerance
            
            print(f"UNID {candidate}:")
            print(f"  Log(E_peak): ANN={ann_epeak:.6f}, OCSVM={ocsvm_epeak:.6f}, diff={epeak_diff:.2e} {'✓' if epeak_match else '✗'}")
            print(f"  Log(beta):   ANN={ann_beta:.6f}, OCSVM={ocsvm_beta:.6f}, diff={beta_diff:.2e} {'✓' if beta_match else '✗'}")
            
            if not (epeak_match and beta_match):
                consistent_data = False
                print(f"  ⚠ INCONSISTENCIA detectada en UNID {candidate}")
            print()
    else:
        print(f"UNID {candidate}: No encontrado en uno de los datasets")
        if len(ann_row) == 0:
            print(f"  - Falta en datos ANN")
        if len(ocsvm_row) == 0:
            print(f"  - Falta en datos OCSVM")
        print()

if consistent_data:
    print("✓ VERIFICACIÓN EXITOSA: Los datos son consistentes entre ANN y OCSVM")
else:
    print("✗ ADVERTENCIA: Se detectaron inconsistencias en los datos")
    print("  Revisar preprocesado y fuentes de datos")


In [ ]:
# ANÁLISIS DE INTERSECCIONES
print("ANÁLISIS DE CONSENSO ENTRE ALGORITMOS\n")

# Outliers OCSVM vs Top ANN
intersection_outliers_ann4 = ocsvm_outliers.intersection(ann_top4)
intersection_outliers_ann10 = ocsvm_outliers.intersection(ann_top10)
intersection_outliers_consensus = ocsvm_outliers.intersection(ann_high_consensus)

print(f"Outliers OCSVM (4) vs ANN:")
print(f"  - vs Top 4 ANN: {len(intersection_outliers_ann4)} común(es) → {sorted(intersection_outliers_ann4)}")
print(f"  - vs Top 10 ANN: {len(intersection_outliers_ann10)} común(es) → {sorted(intersection_outliers_ann10)}")
print(f"  - vs Alto consenso ANN: {len(intersection_outliers_consensus)} común(es) → {sorted(intersection_outliers_consensus)}\n")

In [ ]:
# Top 10 OCSVM vs Top 10 ANN
intersection_top10 = ocsvm_top10.intersection(ann_top10)

print(f"Top 10 vs Top 10:")
print(f"  - Candidatos comunes: {len(intersection_top10)} → {sorted(intersection_top10)}")
print(f"  - Overlap rate: {len(intersection_top10)/10*100:.1f}%\n")

In [ ]:

# CARACTERIZACIÓN DE CANDIDATOS COMUNES
print("CARACTERIZACIÓN DE CANDIDATOS COMUNES\n")

all_common_candidates = intersection_outliers_ann10.union(intersection_top10)

if len(all_common_candidates) > 0:
    print(f"Candidatos con consenso entre algoritmos: {sorted(all_common_candidates)}\n")
    
    for candidate in sorted(all_common_candidates):
        # Info OCSVM
        ocsvm_info = df_unids_results[df_unids_results['number'] == candidate]
        if len(ocsvm_info) > 0:
            ocsvm_rank = ocsvm_info['Anomaly_Rank(%)'].iloc[0]
            ocsvm_score = ocsvm_info['svm_score'].iloc[0]
            is_outlier = ocsvm_info['prediction'].iloc[0] == -1
        else:
            ocsvm_rank, ocsvm_score, is_outlier = 0, 0, False
            
        # Info ANN
        ann_info = df_ann_consensus[df_ann_consensus['Source_ID'] == candidate]
        if len(ann_info) > 0:
            ann_rank = ann_info.index[0] + 1
            ann_consensus = ann_info['Mean_Consensus'].iloc[0]
            ann_std = ann_info['Std_Consensus'].iloc[0]
        else:
            ann_rank, ann_consensus, ann_std = 999, 0, 0
            
        print(f"UNID {candidate}:")
        print(f"  OCSVM: Rank {ocsvm_rank:5.1f}% | Score {ocsvm_score:8.4f} | Outlier: {'Sí' if is_outlier else 'No'}")
        print(f"  ANN:   Pos #{ann_rank:2d}     | Consenso {ann_consensus:5.3f}±{ann_std:.3f}\n")
else:
    print("NO HAY CANDIDATOS COMUNES entre outliers OCSVM y top ANN\n")


In [ ]:
# ANÁLISIS DE DIVERGENCIAS
print("ANÁLISIS DE DIVERGENCIAS\n")

ocsvm_only = ocsvm_outliers - ann_top10
ann_only = ann_top10 - ocsvm_top10

print(f"Candidatos únicos por método:")
print(f"  - Solo OCSVM outliers: {len(ocsvm_only)} → {sorted(ocsvm_only)}")
print(f"  - Solo ANN top 10: {len(ann_only)} → {sorted(ann_only)}\n")


In [ ]:
# POSICIONES CRUZADAS
print("POSICIONES CRUZADAS DE CANDIDATOS\n")

print(f"Posición de outliers OCSVM en ranking ANN:")
for candidate in sorted(ocsvm_outliers):
    ann_info = df_ann_consensus[df_ann_consensus['Source_ID'] == candidate]
    if len(ann_info) > 0:
        ann_rank = ann_info.index[0] + 1
        ann_consensus = ann_info['Mean_Consensus'].iloc[0]
        print(f"  UNID {candidate}: Posición #{ann_rank:3d} en ANN (consenso: {ann_consensus:.3f})")
    else:
        print(f"  UNID {candidate}: No encontrado en resultados ANN")

print(f"\nPosición de top 4 ANN en ranking OCSVM:")
for candidate in sorted(ann_top4):
    ocsvm_info = df_unids_results[df_unids_results['number'] == candidate]
    if len(ocsvm_info) > 0:
        ocsvm_rank = ocsvm_info['Anomaly_Rank(%)'].iloc[0]
        is_outlier = ocsvm_info['prediction'].iloc[0] == -1
        print(f"  UNID {candidate}: Anomaly Rank {ocsvm_rank:5.1f}% ({'Outlier' if is_outlier else 'Inlier'})")
    else:
        print(f"  UNID {candidate}: No encontrado en resultados OCSVM")

In [ ]:
# RESUMEN COMPARATIVO
print(f"\nRESUMEN COMPARATIVO\n")

total_candidates_combined = len(ocsvm_top10.union(ann_top10))
overlap_rate = len(intersection_top10) / 10 * 100

print(f"Métricas de consenso:")
print(f"  - Candidatos únicos combinados: {total_candidates_combined}")
print(f"  - Overlap Top 10 vs Top 10: {len(intersection_top10)}/10 ({overlap_rate:.1f}%)")
print(f"  - Complementariedad: {100-overlap_rate:.1f}%\n")

print(f"Niveles de consenso:")
if len(intersection_outliers_ann4) > 0:
    print(f"  Consenso alto: {len(intersection_outliers_ann4)} candidato(s) detectado(s) por ambos métodos")
else:
    print(f"  Consenso bajo: Sin overlap entre outliers OCSVM y top 4 ANN")

print(f"\nInterpretación:")
if overlap_rate >= 30:
    print(f"  Convergencia significativa entre algoritmos")
elif overlap_rate >= 10:
    print(f"  Convergencia moderada - métodos complementarios")
else:
    print(f"  Divergencia alta - algoritmos capturan anomalías diferentes")

# VERSIÓN 2 - ANTERIOR

In [ ]:
# Cargar resultados del consenso ANN 2F
ann_consensus = pd.read_csv('../data/results/ann/2F/consensus_analysis_ann_2f.csv')
print(f"Resultados ANN 2F cargados: {len(ann_consensus)} fuentes")

# Usar los resultados OCSVM con predicciones (NO df_unids original)
ocsvm_results = df_unids_results.copy()  # Ya tiene las predicciones y scores
print(f"Resultados OCSVM 2F cargados: {len(ocsvm_results)} fuentes")

# Verificar mismo número de fuentes
if len(ann_consensus) != len(ocsvm_results):
    print(f" ADVERTENCIA: Diferentes números de fuentes - ANN: {len(ann_consensus)}, OCSVM: {len(ocsvm_results)}")
else:
    print(f"Mismo número de fuentes: {len(ann_consensus)}")

In [ ]:
print("Resultados (top unids) consenso ANN 2F:")
ann_consensus.head()

In [ ]:
print('Resultados (todos) OCSVM 2F:')
ocsvm_results.head()

In [ ]:
# Verificar alineamiento de IDs
ann_ids = ann_consensus['Source_ID'].values
ocsvm_ids = ocsvm_results['number'].values if 'number' in ocsvm_results.columns else ocsvm_results.index.values

ids_aligned = np.array_equal(ann_ids, ocsvm_ids)

if ids_aligned:
    print("Los IDs están alineados correctamente")
    ann_final = ann_consensus.copy()
    ocsvm_final = ocsvm_results.copy()
else:
    print("Los IDs NO están alineados - procediendo a alinear...")
    print(f"   Primeros 5 IDs ANN: {ann_ids[:5]}")
    print(f"   Primeros 5 IDs OCSVM: {ocsvm_ids[:5]}")
    
    # Alinear por ID común
    if 'number' in ocsvm_results.columns:
        ann_final = ann_consensus.sort_values('Source_ID').reset_index(drop=True)
        ocsvm_final = ocsvm_results.sort_values('number').reset_index(drop=True)
        
        # Verificar alineamiento después de ordenar
        if np.array_equal(ann_final['Source_ID'].values, ocsvm_final['number'].values):
            print("IDs alineados correctamente después de ordenar")
        else:
            print("ERROR: No se pudieron alinear los IDs - revisar datos")
            # Mostrar diferencias
            ann_set = set(ann_final['Source_ID'].values)
            ocsvm_set = set(ocsvm_final['number'].values)
            print(f"   IDs solo en ANN: {ann_set - ocsvm_set}")
            print(f"   IDs solo en OCSVM: {ocsvm_set - ann_set}")
    else:
        print("ERROR: Columna 'number' no encontrada en OCSVM results")
        ann_final = ann_consensus.copy()
        ocsvm_final = ocsvm_results.copy()

In [ ]:
# CREAR DATAFRAME COMBINADO

# Verificar columnas necesarias existen
required_ann_cols = ['Source_ID', 'E_peak', 'Beta', 'log_E_peak', 'log_Beta', 'Mean_Prob_Consensus', 'Std_Prob_Consensus']
required_ocsvm_cols = ['prediction', 'Anomaly_Score', 'Anomaly_Rank(%)']

missing_ann = [col for col in required_ann_cols if col not in ann_final.columns]
missing_ocsvm = [col for col in required_ocsvm_cols if col not in ocsvm_final.columns]

if missing_ann:
    print(f"Columnas faltantes en ANN: {missing_ann}")
if missing_ocsvm:
    print(f"Columnas faltantes en OCSVM: {missing_ocsvm}")

if not missing_ann and not missing_ocsvm:
    comparison_df = pd.DataFrame({
        # IDs y características originales
        'Source_ID': ann_final['Source_ID'].values,
        'E_peak': ann_final['E_peak'].values,
        'Beta': ann_final['Beta'].values,
        'Log_E_peak': ann_final['log_E_peak'].values,
        'Log_Beta': ann_final['log_Beta'].values,
        
        # Resultados ANN 2F
        'ANN_Prob_Mean': ann_final['Mean_Prob_Consensus'].values,
        'ANN_Prob_Std': ann_final['Std_Prob_Consensus'].values,
        
        # Resultados OCSVM 2F
        'OCSVM_Prediction': ocsvm_final['prediction'].values,  # 1=inlier, -1=outlier
        'OCSVM_Anomaly_Score': ocsvm_final['Anomaly_Score'].values,
        'OCSVM_Anomaly_Rank': ocsvm_final['Anomaly_Rank(%)'].values,
    })
    
else:
    print("No se pudo crear el DataFrame combinado debido a columnas faltantes")
    comparison_df = None

In [ ]:
# DIAGNÓSTICO DIRECTO ANN 2F vs OCSVM 2F
print("DIAGNÓSTICO ANN 2F vs OCSVM 2F")

print("-" * 60)
print(f"ANN 2F - Media: {comparison_df['ANN_Prob_Mean'].mean():.4f} | "
      f"Mediana: {comparison_df['ANN_Prob_Mean'].median():.4f} | "
      f"Rango: [{comparison_df['ANN_Prob_Mean'].min():.4f}-{comparison_df['ANN_Prob_Mean'].max():.4f}]")

print(f"OCSVM 2F - Media: {comparison_df['OCSVM_Anomaly_Rank'].mean():.1f}% | "
      f"Mediana: {comparison_df['OCSVM_Anomaly_Rank'].median():.1f}% | "
      f"Rango: [{comparison_df['OCSVM_Anomaly_Rank'].min():.1f}%-{comparison_df['OCSVM_Anomaly_Rank'].max():.1f}%]")

# IDENTIFICACIÓN DE CANDIDATOS
print("-" * 60)
print(f"\nIDENTIFICACIÓN DE CANDIDATOS:")
ann_threshold = np.percentile(comparison_df['ANN_Prob_Mean'], 95)
ocsvm_threshold = np.percentile(comparison_df['OCSVM_Anomaly_Rank'], 95)

top_ann = comparison_df[comparison_df['ANN_Prob_Mean'] >= ann_threshold].sort_values('ANN_Prob_Mean', ascending=False)
ocsvm_outliers = comparison_df[comparison_df['OCSVM_Prediction'] == -1].sort_values('OCSVM_Anomaly_Rank', ascending=False)
top_ocsvm = comparison_df[comparison_df['OCSVM_Anomaly_Rank'] >= ocsvm_threshold].sort_values('OCSVM_Anomaly_Rank', ascending=False)

print(f"Top ANN (P95={ann_threshold:.4f}): {len(top_ann)} fuentes")
print(f"OCSVM Outliers: {len(ocsvm_outliers)} fuentes")
print(f"Top OCSVM (P95={ocsvm_threshold:.1f}%): {len(top_ocsvm)} fuentes")

# SOLAPAMIENTOS
print(f"\nSOLAPAMIENTOS:")
overlap_restrictivo = set(top_ann['Source_ID']) & set(ocsvm_outliers['Source_ID'])
overlap_amplio = set(top_ann['Source_ID']) & set(top_ocsvm['Source_ID'])

print(f"Top ANN ∩ OCSVM Outliers: {len(overlap_restrictivo)} fuentes")
if overlap_restrictivo:
    print(f"  IDs: {sorted(list(overlap_restrictivo))}")

print(f"Top ANN ∩ Top OCSVM: {len(overlap_amplio)} fuentes")
if overlap_amplio:
    print(f"  IDs: {sorted(list(overlap_amplio))}")

# TOP 10 COMPARACIÓN DIRECTA
print(f"\nTOP 10 ANN vs OCSVM:")
print("-" * 60)
for i, (_, row) in enumerate(top_ann.head(10).iterrows()):
    status = "Outlier" if row['OCSVM_Prediction'] == -1 else "Normal"
    print(f"{i+1:2d}. ID {int(row['Source_ID']):4d}: ANN={row['ANN_Prob_Mean']:.4f} | OCSVM={row['OCSVM_Anomaly_Rank']:5.1f}% {status}")

print(f"\nOUTLIERS OCSVM vs ANN:")
print("-" * 60)
if len(ocsvm_outliers) > 0:
    for i, (_, row) in enumerate(ocsvm_outliers.iterrows()):
        print(f"{i+1:2d}. ID {int(row['Source_ID']):4d}: OCSVM={row['OCSVM_Anomaly_Rank']:5.1f}% | ANN={row['ANN_Prob_Mean']:.4f}")
else:
    print("No hay outliers detectados por OCSVM")

# ANÁLISIS DE CONCORDANCIA
print(f"\nANÁLISIS DE CONCORDANCIA:")
# Correlación entre métricas
correlation = np.corrcoef(comparison_df['ANN_Prob_Mean'], comparison_df['OCSVM_Anomaly_Rank'])[0, 1]
print(f"Correlación ANN-OCSVM: {correlation:.4f}")

# Concordancia en clasificación binaria
ann_high = comparison_df['ANN_Prob_Mean'] >= ann_threshold
ocsvm_high = comparison_df['OCSVM_Anomaly_Rank'] >= ocsvm_threshold

concordancia = np.mean(ann_high == ocsvm_high) * 100
print(f"Concordancia en top 5%: {concordancia:.1f}%")

# Casos discrepantes
discrepantes_ann_high_ocsvm_low = comparison_df[(comparison_df['ANN_Prob_Mean'] >= ann_threshold) & 
                                               (comparison_df['OCSVM_Anomaly_Rank'] < 50)]
discrepantes_ocsvm_high_ann_low = comparison_df[(comparison_df['OCSVM_Anomaly_Rank'] >= ocsvm_threshold) & 
                                               (comparison_df['ANN_Prob_Mean'] < 0.5)]

print(f"Discrepantes (ANN alta, OCSVM baja): {len(discrepantes_ann_high_ocsvm_low)}")
print(f"Discrepantes (OCSVM alta, ANN baja): {len(discrepantes_ocsvm_high_ann_low)}")

print("-" * 60)
# RESUMEN EJECUTIVO
print(f"\nRESUMEN EJECUTIVO:")
if len(overlap_restrictivo) > 0:
    print(f"Consenso encontrado: {len(overlap_restrictivo)} fuentes identificadas por ambos métodos")
else:
    print("Sin consenso directo entre métodos")

if abs(correlation) > 0.5:
    print(f"Correlación moderada-alta: {correlation:.3f}")
else:
    print(f"Correlación baja: {correlation:.3f}")

if concordancia > 70:
    print(f"Buena concordancia en top candidatos: {concordancia:.1f}%")
else:
    print(f"Baja concordancia en top candidatos: {concordancia:.1f}%")